In [1]:
pip install pandas numpy matplotlib seaborn plotly dash streamlit

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Required libraries install karo
# !pip install pandas numpy matplotlib seaborn plotly dash streamlit

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from dash import Dash, dcc, html, Input, Output

# Dataset ko load karo
file_path = r'C:\Users\Nidhi Verma\grammy\grammy_awards.csv'
df = pd.read_csv(file_path)

# Dataset ko check karo
print(df.columns)

# Unnecessary columns ko hatane ke bajaye sirf relevant columns rakho
df = df[['year', 'category', 'artist', 'song_or_album', 'winner']]

# 'song_or_album' ko split karo (if applicable)
df[['song', 'album']] = df['song_or_album'].str.split('-', n=1, expand=True)

# Missing values ko fill karo (FutureWarning avoid karne ke liye)
df['winner'] = df['winner'].fillna(False)
df['category'] = df['category'].fillna('Unknown')
df['artist'] = df['artist'].fillna('Unknown')

# Dashboard create karo
app = Dash(__name__)

app.layout = html.Div([
    html.H1("Grammy Awards Analysis Dashboard", style={'textAlign': 'center'}),

    # Dropdown for selecting year
    dcc.Dropdown(
        id='year-dropdown',
        options=[{'label': year, 'value': year} for year in sorted(df['year'].unique())],
        value=df['year'].min(),
        multi=False,
        style={'width': '50%', 'margin': 'auto'}
    ),

    # Graph for Award trends by year
    dcc.Graph(id='award-trend'),

    # Graph for Most awarded artist
    dcc.Graph(id='most-awarded-artist'),

    # Graph for Most popular genre
    dcc.Graph(id='most-popular-genre'),

    # Graph for Male vs Female award ratio (if available)
    dcc.Graph(id='gender-ratio')
])

# Callbacks define karo
@app.callback(
    Output('award-trend', 'figure'),
    Output('most-awarded-artist', 'figure'),
    Output('most-popular-genre', 'figure'),
    Output('gender-ratio', 'figure'),
    Input('year-dropdown', 'value')
)
def update_dashboard(selected_year):
    # 🎯 Award trends by year
    trend_df = df[df['year'] == selected_year]
    trend_fig = px.histogram(trend_df, x='category', title=f"Award Trends in {selected_year}")

    # 🎯 Most awarded artist
    artist_count = df['artist'].value_counts().nlargest(10).reset_index()
    artist_count.columns = ['artist', 'count']
    artist_fig = px.bar(artist_count, x='artist', y='count', title="Most Awarded Artists")

    # 🎯 Most popular genre
    genre_count = df['category'].value_counts().nlargest(10).reset_index()
    genre_count.columns = ['category', 'count']
    genre_fig = px.bar(genre_count, x='category', y='count', title="Most Popular Genre")

    # 🎯 Male vs Female ratio (if gender data available)
    if 'gender' in df.columns:
        gender_count = df['gender'].value_counts().reset_index()
        gender_count.columns = ['gender', 'count']
        gender_fig = px.pie(gender_count, names='gender', values='count', title="Male vs Female Award Ratio")
    else:
        gender_fig = px.pie(title="Gender data not available")

    return trend_fig, artist_fig, genre_fig, gender_fig

# Server ko run karo
if __name__ == '__main__':
    app.run(debug=True)


Index(['year', 'annual_edition', 'category', 'artist', 'song_or_album',
       'winner', 'url'],
      dtype='object')
